In [2]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [3]:
import random

REASONS = ("Didn't order", "Change flight date", "Busy on that day", "Refund", "Flight delayed too many times", "I regret", "I broke my legs", "I was drunk", "Grandpa stroke")
REASONS_WEIGHTS = (15, 4, 1, 5, 2, 1, 1, 1, 10)
def get_random_reason():
    return random.sample(REASONS, k=1, counts=REASONS_WEIGHTS)[0]

In [11]:
from typing import TypedDict
from datetime import datetime

stmt = "SELECT PAYMENT_ID, CREATED_AT FROM PAYMENT"

class Payment(TypedDict):
    id: str
    created_at: datetime
    

PICKED_PAYMENT = [Payment(id=row[0], created_at=row[1]) for row in cur.execute(stmt) if random.randint(1, 100) == 1]

In [ ]:
from datetime import timedelta

def deviate_time(dt: datetime, max_days: int) -> datetime:
    return dt + timedelta(days=random.randint(0, max_days), hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))

In [10]:
from utils import paginate_insert_all

sql = "    INTO (PAYMENT_ID, CREATED_AT, REASON) VALUES ('{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), '{}') "
@paginate_insert_all
def insert_refund(payment_id: str, refund_dt: datetime, refund_reason: str):
    print(sql.format(payment_id, refund_dt, refund_reason))
    
for payment in PICKED_PAYMENT:
    insert_refund(payment.id, deviate_time(payment.created_at, 7), get_random_reason())
print("SELECT 1 FROM DUAL;")

136

In [ ]:
con.close()